In [ ]:
import requests, json, re, urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
import wget,gzip
from pathlib import Path
import numpy as np
import yaml



## Source and normalize movie / TV show data ##
### IMdB ###
- use batch interface: https://datasets.imdbws.com
- subset the data based on a list of actors and titles

### OMdB ###
- use API : http://www.omdbapi.com/?i=tt3896198&apikey=xxx
- the key is *imdb_id* so simple,  but rate limited to 1000 requests a day
- attempt to source based on interesting titles subset from **IMdB**

### TMdB ###
- use API: https://api.themoviedb.org/3/movie/{id}   No limits.... but latency
- the key is it's own... get batch file: https://developers.themoviedb.org/3/getting-started/daily-file-exports
- join on *title* to **IMdB** subset to get wanted list to retrieve.  Result includes *imdb_id*

### apple - includes cost data ###
- use API: https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/Searching.html#//apple_ref/doc/uid/TP40017632-CH5-SW1
- this is rate limited to 20 calls per minute.  Hence sleep capability
- add search, tconst and name back to data returned so it's simple to identify and join

### boxofficemojo - includes boxoffice data ###
- BeautifulSoup web scrape https://www.boxofficemojo.com/title/tt0021106/?ref_=bo_se_r_1
- the tconst is included in the URL so simple to search...  just painful to scrape
- add search and tconst to scraped data so it's easy to identify and join

In [ ]:
import movies.utils

# a bit of discipline on where files are and secrets...
p = Path().cwd().joinpath("data")
with open ("../apikeys.yaml") as f: keys = yaml.safe_load(f)

# l = 0 means just get.... so can use separete python process to be busy sourcing...
l = 0
    
files = movies.utils.imdb.getdata(p)
# regen subset...
if False:
    dfs = movies.utils.imdb.normalise(files, alldata=True, subsetdata=True)
dfs = movies.utils.imdb.normalise(files, alldata=False, subsetdata=True)
dfs, files = movies.utils.omdb.getandnormalise(keys["keys"]["omdb"], dfs, files, path=p, limit=l)
dfs, files = movies.utils.tmdb.getandnormalise(keys["keys"]["tmdb"], dfs, files, path=p, limit=l)
dfs, files = movies.utils.apple.getandnormalise(dfs, files, path=p, limit=l, sleep=3)
dfs, files = movies.utils.mojo.getandnormalise(dfs, files, path=p, limit=l)

# remind ourselves of available data...
dfs.keys()

In [ ]:
# The Dark Knight	tt0468569	
# Game of Throne tt0944947
# for demo purpose - just pick first association when there are many
mask = dfs["title.basics"]["tconst"].isin(["tt0468569","tt0944947"])
mask = dfs["title.basics"].index # everything... not reset_index() to make it good
# mask = dfs["omdb.titles"]["Type"]=="movie"

# utility functions to understand source of data from column names
def renamec(df, prefix="tbd_", customExc=[]):
    exc = ["tconst"] if len(customExc)==0 else customExc
    proxy = {"imdbID":"tconst", "imdb_id":"tconst"}
    return {"columns":{c:f"{prefix}{c}" if c not in proxy.keys() else proxy[c]
                       for c in df.columns if c not in exc}}
def cleandf(dfs, key, prefix, customExc=[]):
    return dfs[key].rename(**renamec(dfs[key], prefix, customExc))

demo = (dfs["title.basics"].loc[mask]
 .rename(**renamec(dfs["title.basics"], "tb_"))
 .merge(cleandf(dfs, "omdb.titles", "omdb_"), on="tconst", how="left")
 .merge(cleandf(dfs, "title.ratings", "tr_"), on="tconst", how="left")
 .merge(cleandf(dfs, "title.akas", "alias_").groupby("tconst", as_index=False).first(), on="tconst", how="left")
 .merge(cleandf(dfs, "title.crew", "crew_").groupby("tconst", as_index=False).first(), on="tconst")
 .merge(cleandf(dfs, "title.principals", "ppl_").groupby("tconst", as_index=False).first(), on="tconst", how="left")
 .merge(cleandf(dfs, "title.episode", "ep_", ["x"]).groupby("ep_parentTconst", as_index=False).first(), 
        left_on="tconst", right_on="ep_parentTconst", how="left")
 .merge(dfs["nmi"]
        .merge(cleandf(dfs, "name.basics", "nb_", ["tconst","nconst"]), on="nconst")
        .groupby(["tconst"], as_index=False).first(), on="tconst", suffixes=("","_name"))
 .merge(cleandf(dfs, "tmdb.titles", "tmdb_"), on="tconst", how="left")
#  .merge(cleandf(dfs, "titles.apple", "itunes_").query("itunes_strongmatch==True").groupby("tconst", as_index=False).first(), on="tconst", how="left")
 .merge(cleandf(dfs, "titles.apple", "itunes_")
        .sort_values(["tconst","itunes_strongmatch","itunes_trackId"], ascending=[True,False,True])
        .groupby("tconst", as_index=False).first(), on="tconst", how="left")
 .merge(cleandf(dfs, "mojo.boxoffice", "mojo_").groupby("tconst", as_index=False).first(), on="tconst", how="left")


)
pd.options.display.max_rows=200
pd.options.display.max_columns=200
cols = movies.utils.util.catcolumns(demo)
(demo
#  .dropna(subset=["tb_genres"])
#  .query("tb_genres.str.contains('Romance') & tb_genres.str.contains('Comedy')")
#  .sort_values(["itunes_trackPrice","mojo_AllWorldwide"], ascending=False)
 .sort_values(["mojo_AllWorldwide"], ascending=False)
 .loc[:,cols.query("~type.isin(['url','artwork','list'])").index]
 .head(5)
#  .to_html(p.joinpath("rc.html"), index=False)

)
# TODO beautifulsoup it to insert <div> elements so don't need javascript enabled browser
# with open(p.joinpath("rc.html"), "a") as f, open(p.parent.parent.joinpath("inject.html")) as f2: 
#     f.write(f2.read())

### data quality ###
1. what columns are URLs / links to files
2. what columns are USD but as strings

In [ ]:
# dfs["titles.apple"].query("tconst=='tt0499549'")
# dfs["tmdb.titles"].query("imdb_id=='tt0499549'")

movies.utils.util.catcolumns(dfs["mojo.boxoffice"]).query("type=='float64'").index.tolist()
moneycols=['AllDomestic','AllInternational','AllWorldwide','Domestic','International','Worldwide','Opening','Budget','Domestic Opening']
badlist = dfs["mojo.boxoffice"].query("~search.str.endswith('credits/')").groupby(["tconst"]).agg({c:lambda x: x.isna().all() for c in moneycols})
# badlist = dfs["mojo.boxoffice"].groupby(["tconst"]).agg({c:lambda x: x.isna().all() for c in moneycols})

bt = badlist.T.all().to_frame().rename(columns={0:"notok"}).query("notok")
bt = (bt.merge(dfs["title.ratings"], on="tconst")
      .merge(dfs["title.basics"], on="tconst")
      .sort_values(["startYear","numVotes"], ascending=[False, False])
      .query("numVotes>10000 and startYear>=1975 and runtimeMinutes>=60")
     )
bt
# di = dfs["mojo.boxoffice"].loc[dfs["mojo.boxoffice"]["tconst"].isin(bt["tconst"])].index
# di
# dfs["mojo.boxoffice"].drop(di, inplace=True)
# dfs["mojo.boxoffice"].reset_index(drop=True).to_json(files["mojo.boxoffice"])



In [ ]:

if False:
    dfr = pd.read_csv(p.joinpath("title.ratings.tsv.gz"), sep="\t").replace({"\\N":np.nan})
    mask = ~dfr["tconst"].isin(dfs["title.basics"]["tconst"])
#     dfr.loc[mask,].sort_values(["numVotes","averageRating"], ascending=False).head(10)

    pd.concat([pd.read_json(p.joinpath("wanted.json")),
               dfr.loc[mask,].sort_values(["numVotes","averageRating"], ascending=False).head(20)
              ]).drop_duplicates().reset_index(drop=True).to_json(p.joinpath("wanted.json"), orient="records")

with open(p.joinpath("wanted.json")) as f: tm = json.load(f)
print(len(tm))


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(3,2, figsize=[20,10],
                      sharey=False, sharex=False, gridspec_kw={"hspace":0.3})

# REVENUE percentiles
(demo.dropna(subset=["tb_startYear","mojo_AllWorldwide"])
 .groupby("tb_startYear")
 .agg({"mojo_AllWorldwide":[np.median, lambda x: np.percentile(x, q=95),lambda x: np.percentile(x, q=5)]})
 .droplevel(0, axis=1)
 .rename(columns={"<lambda_0>":"95th","<lambda_1>":"5th"})
 .plot(ax=ax[0][0])
)
ax[0][0].set_xlabel("Worldwide revenue", weight='bold', fontsize=12)

# TITLES summary
various = ['tvEpisode','short','tvShort','tvSpecial','video','videoGame','tvMovie','tvMiniSeries']
(demo.dropna(subset=["tb_startYear","tb_titleType","tb_primaryTitle"])
#  .query("~tb_titleType.isin(['tvEpisode','short','tvShort','tvSpecial','video','videoGame','tvMovie','tvMiniSeries'])")
 .assign(decade=lambda x: np.where(x["tb_startYear"]<1970, 1960, (x["tb_startYear"]//10)*10),
        tb_titleType=lambda x: np.where(x["tb_titleType"].isin(various), "various", x["tb_titleType"]))
 .groupby(["tb_titleType","decade"])
 .agg({"tb_primaryTitle":["count"]})
 .unstack(0)
 .droplevel([0,1], axis=1)
 .plot(ax=ax[0][1], kind="bar")
)
ax[0][1].set_xlabel("Titles by decade", weight='bold', fontsize=12)
for tick in ax[0][1].get_xticklabels(): tick.set_rotation(0)

# WORLDWIDE revenu
cap = 1.75e9
cap = np.percentile(demo["mojo_AllWorldwide"].dropna(), q=99.9)
(demo.dropna(subset=["mojo_AllWorldwide","itunes_trackPrice","tr_numVotes"])
 .assign(tb_startYear=lambda x: np.where(x["tb_startYear"]<2000, 2000, x["tb_startYear"]).astype("float64"),
        itunes_trackPrice=lambda x: x["itunes_trackPrice"].astype("float64"))
 .assign(mojo_AllWorldwide=lambda x: np.where(x["mojo_AllWorldwide"]>cap,cap,x["mojo_AllWorldwide"]))
 .plot(ax=ax[1][1], x="mojo_AllWorldwide", y="tr_numVotes", s="itunes_trackPrice", c="tb_startYear", kind="scatter", colormap="jet")
)
ax[1][1].set_xlabel(f"revenue vs votes, year and price Limit:{cap/1e9:.2f}B", weight='bold', fontsize=12)

# iTunes price
(demo.dropna(subset=["tb_startYear","itunes_trackPrice"])
 .groupby("tb_startYear")
 .agg({"itunes_trackPrice":[np.median, lambda x: np.percentile(x, q=95),lambda x: np.percentile(x, q=5)]})
 .droplevel(0, axis=1)
 .rename(columns={"<lambda_0>":"95th","<lambda_1>":"5th"})
 .plot(ax=ax[1][0])
)
ax[1][0].set_xlabel("iTunes price", weight='bold', fontsize=12)

# ATTRIBUTE summary
(pd.concat([dfs[k].count().to_frame().rename(columns={0:k}) for k in dfs.keys()])
 .T.plot(ax=ax[2][0], kind="barh", stacked=True)
)
ax[2][0].get_legend().remove()
ax[2][0].set_xlabel(f"Attribute Summary", weight='bold', fontsize=12)

# MARKET revenue
cap = np.percentile(dfs["mojo.boxoffice"]["Worldwide"].dropna(), q=90)
df = (dfs["mojo.boxoffice"].dropna(subset=["Worldwide","Markets"])
 .merge(cleandf(dfs, "title.ratings", "tr_"), on="tconst", how="left")
 .assign(Markets=lambda x: x["Markets"].str.replace("^([0-9]+)(.*)$",r"Multi \2"))
 .assign(MarketsF=lambda x: pd.factorize(x["Markets"])[0].astype("float64"))
# rebase so it's in same range as number of markets
 .assign(tr_numVotes=lambda x: (x["tr_numVotes"]/x["tr_numVotes"].max())*x["MarketsF"].max() )
 .merge(dfs["title.basics"], on="tconst")
 .assign(startYear=lambda x: np.where(x["startYear"]<2000, 2000, x["startYear"]).astype("float64"))
 .assign(Worldwide=lambda x: np.where(x["Worldwide"]>cap,cap,x["Worldwide"]))
 
)
df.plot(ax=ax[2][1], kind="scatter", y="MarketsF", x="Worldwide", c="startYear", s="tr_numVotes", colormap="jet")
# build table to decode y-axis
dft = (df.groupby(["MarketsF","Markets"], as_index=False)["Worldwide"]
       .count()
       .sort_values("MarketsF", ascending=True)
       .reset_index(drop=True)
       .assign(Markets=lambda x: x.apply(lambda r: f'{r.Markets} ({r.Worldwide})', axis=1))
       .drop(columns="Worldwide")
       .astype("int64", errors="ignore")
      )
dft = pd.concat([dft.loc[i*10:(i*10)+9].reset_index(drop=True) for i in range(7)], axis=1)
ax[2][1].set_xlabel(f"Market vs Revenue. Colour:Year, Size:Votes Limit:{cap/1e6:.2f}M", weight='bold', fontsize=12)
ax[2][1].table(dft.values,
        colWidths=[0.02,.14] * 7,
         bbox=[0, -0.65, (0.02+.14)*7, 0.4],
        cellLoc="left"
        )
# MARKET revenue *end*



In [ ]:

fig, ax = plt.subplots(1, figsize=[20,5],
                      sharey=False, sharex=False, gridspec_kw={"hspace":0.5})


cap = .1e9
cap = np.percentile(dfs["mojo.boxoffice"]["Worldwide"].dropna(), q=90)
df = (dfs["mojo.boxoffice"].dropna(subset=["Worldwide","Markets"])
 .merge(cleandf(dfs, "title.ratings", "tr_"), on="tconst", how="left")
 .assign(Markets=lambda x: x["Markets"].str.replace("^([0-9]+)(.*)$",r"Multi \2"))
 .assign(MarketsF=lambda x: pd.factorize(x["Markets"])[0].astype("float64"))
# rebase so it's in same range as number of markets
 .assign(tr_numVotes=lambda x: (x["tr_numVotes"]/x["tr_numVotes"].max())*x["MarketsF"].max() )
 .merge(dfs["title.basics"], on="tconst")
 .assign(startYear=lambda x: np.where(x["startYear"]<2000, 2000, x["startYear"]).astype("float64"))
 .assign(Worldwide=lambda x: np.where(x["Worldwide"]>cap,cap,x["Worldwide"]))
 
)
df.plot(ax=ax, kind="scatter", y="MarketsF", x="Worldwide", c="startYear", s="tr_numVotes", colormap="jet")
# build table to decode y-axis
dft = (df.groupby(["MarketsF","Markets"], as_index=False)["Worldwide"]
       .count()
       .sort_values("MarketsF", ascending=True)
       .reset_index(drop=True)
       .assign(Markets=lambda x: x.apply(lambda r: f'{r.Markets} ({r.Worldwide})', axis=1))
       .drop(columns="Worldwide")
       .astype("int64", errors="ignore")
      )
dft = pd.concat([dft.loc[i*10:(i*10)+9].reset_index(drop=True) for i in range(7)], axis=1)
ax.set_xlabel(f"Market vs Revenue. Colour:Year, Size:Votes Limit:{cap/1e6:.2f}M", weight='bold', fontsize=12)
ax.table(dft.values,
        colWidths=[0.02,.14] * 7,
         bbox=[0, -0.55, (0.02+.14)*7, 0.4],
        cellLoc="left"
        )

# dft

In [ ]:
fig, ax = plt.subplots(1, figsize=[20,5],
                      sharey=False, sharex=False, gridspec_kw={"hspace":0.5})


(pd.concat([dfs[k].count().to_frame().rename(columns={0:k}) for k in dfs.keys()])
 .T.plot(ax=ax, kind="barh", stacked=True)
)
ax.get_legend().remove()
ax.set_xlabel(f"Attribute Summary", weight='bold', fontsize=12)

